In [ ]:
import os
import sys
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.DEBUG)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q ruamel.yaml
    !pip install -q tensorboard-plugin-profile
    project_path = '/content/drive/MyDrive/Colab Projects/QuantumFlow'
except:
    project_path = os.path.expanduser('~/QuantumFlow')

In [ ]:
os.chdir(project_path)
sys.path.append(project_path)

import numpy as np
import tensorflow as tf
import tree

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import quantumflow

experiment = 'crazynet'
run_name = 'default_while_loop'

base_dir = os.path.join(project_path, "experiments", experiment)
params = quantumflow.utils.load_yaml(os.path.join(base_dir, 'hyperparams.yaml'))[run_name]
run_dir = os.path.join(base_dir, run_name)

In [ ]:
dataset_train = quantumflow.instantiate(params['dataset_train'], run_dir=run_dir)
dataset_train.build()

dataset_validate = quantumflow.instantiate(params['dataset_validate'], run_dir=run_dir)
dataset_validate.build()

In [ ]:
crazynet = quantumflow.instantiate(params['model'], num_outputs=1)

In [ ]:
x = np.zeros(shape=(100, 500, 1), dtype=np.float32)
x_inputs = np.zeros(shape=(100, 500, 500, 1), dtype=np.float32)
inputs = np.zeros(shape=(100, 500, 500, 1), dtype=np.float32)

In [ ]:
def loop_over_second_batch(core, all_x, all_x_inputs, all_inputs):
    num_x = tf.shape(all_x)[1]
    
    x = tf.TensorArray(all_x.dtype, num_x).unstack(tf.transpose(all_x, perm=[1, 0, 2]))
    x_inputs = tf.TensorArray(all_x_inputs.dtype, num_x).unstack(tf.transpose(all_x_inputs, perm=[1, 0, 2, 3]))
    inputs = tf.TensorArray(all_inputs.dtype, num_x).unstack(tf.transpose(all_inputs, perm=[1, 0, 2, 3]))

    output = core(x.read(0), x_inputs.read(0), inputs.read(0))
    
    output_x = tf.TensorArray(outputs.dtype, num_x).write(0, output)

    for t in tf.range(1, num_x):
        #tf.autograph.experimental.set_loop_options(
        #    parallel_iterations=parallel_iterations,
        #    swap_memory=swap_memory,
        #    maximum_iterations=num_steps - 1)
        
        output = core(x.read(t), x_inputs.read(t), inputs.read(t))
        output_x = output_x.write(t, output)

    all_output = tf.TensorArray.stack(output_x)
    return all_output

In [ ]:
loop_over_second_batch(crazynet, x, x_inputs, inputs)